In [1]:
import pandas as pd

df = pd.read_csv("https://ec.europa.eu/eurostat/databrowser-backend/api/extraction/1.0/LIVE/true/sdmx/csv/HBS_STR_T223?i")


In [2]:
# Keep only the rows where TIME_PERIOD is equal to 2015
df = df[df['TIME_PERIOD'] == 2015]


In [3]:
import pandas as pd

# this file is around 200 mbs so it can take some time to download

df_inflation = pd.read_csv("https://ec.europa.eu/eurostat/databrowser-backend/api/extraction/1.0/LIVE/true/sdmx/csv/PRC_HICP_MANR?i")


In [4]:
# Convert the TIME_PERIOD column to datetime
# df_inflation['TIME_PERIOD'] = pd.to_datetime(df_inflation['TIME_PERIOD'])



In [5]:
# Filter the rows that have a date value later than 2021-03
df_inflation = df_inflation.loc[df_inflation['TIME_PERIOD'] > '2021-03']

In [6]:
df_inflation_pivot = df_inflation.pivot(index=['coicop', 'geo'], columns='TIME_PERIOD', values='OBS_VALUE').reset_index()


In [7]:
merged_df = df.merge(df_inflation_pivot, on=["coicop", "geo"])


In [8]:
merged_df.columns

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'quantile', 'coicop', 'unit', 'geo',
       'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG', '2021-04', '2021-05', '2021-06',
       '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
       '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
       '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
       '2023-01', '2023-02', '2023-03', '2023-04'],
      dtype='object')

In [9]:
merged_df = merged_df.drop(columns=['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'OBS_FLAG', 'TIME_PERIOD'])

In [10]:
cols_to_multiply = [col for col in merged_df.columns if col.startswith('202')]
merged_df[['w' + col for col in cols_to_multiply]] = merged_df[cols_to_multiply].apply(lambda x: x * merged_df['OBS_VALUE']/1000)


In [11]:
merged_df_5_chars = merged_df[merged_df['coicop'].str.len() == 5]


In [12]:
df_inflation_by_quintiles = merged_df_5_chars.groupby(['geo','quantile'])['w2022-03'].sum().reset_index(name='2022-03')

In [13]:
columns_to_sum = [col for col in merged_df_5_chars.columns if col.startswith('w')]
                  
summed_columns = []
for col in columns_to_sum:
    new_col_name = col[1:]
    summed_columns.append(new_col_name)
    merged_df_5_chars[new_col_name] = merged_df_5_chars.groupby(['geo', 'quantile'])[col].transform('sum')

df_inflation_by_quintile = merged_df_5_chars[['geo', 'quantile'] + summed_columns].drop_duplicates()

C:\Users\maxen\AppData\Local\Temp\ipykernel_16700\899747291.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_5_chars[new_col_name] = merged_df_5_chars.groupby(['geo', 'quantile'])[col].transform('sum')
C:\Users\maxen\AppData\Local\Temp\ipykernel_16700\899747291.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_5_chars[new_col_name] = merged_df_5_chars.groupby(['geo', 'quantile'])[col].transform('sum')
C:\Users\maxen\AppData\Local\Temp\ipykernel_16700\899747291.py:7: SettingWithCopyW